In [4]:
import numpy as np
from sklearn.model_selection import ParameterGrid

class CVPR:
    def __init__(self,graph:np.array,demande:list[int],n_vehicules:int,capacity:int):
        self.graph = graph
        self.demande = demande
        self.n_vehicules = n_vehicules
        self.n_points = len(self.graph)
        self.capacity = capacity
        self.pheromone = np.ones((self.n_points, self.n_points))


    def optimize_params(self, n_ants: int = 50, evaporation_rate: float = 0.9, Q: float = 1):
        best_length = float('inf')
        best_params = {'alpha': None, 'beta': None,'iterations' : None}
        iteration = 0

        # Define the parameter grid for alpha and beta
        param_grid = {'alpha': np.linspace(0, 1, 10), 'beta': np.linspace(0, 1, 10),'n_iterations': np.linspace(5,10,6)}

        # Iterate over all combinations of alpha and beta
        for params in ParameterGrid(param_grid):
            iteration = iteration + 1
            print(iteration)
            # Run optimization with current parameters
            _, length = self.ant_colony_optimization(n_ants=n_ants, n_iterations=int(params['n_iterations']), alpha=params['alpha'], beta=params['beta'], evaporation_rate=evaporation_rate, Q=Q)

            # Update best parameters if current solution is better
            if length < best_length:
                best_length = length
                print(best_length)
                best_params['alpha'] = params['alpha']
                best_params['beta'] = params['beta']
                best_params['iterations'] = params['n_iterations']

        return best_params

    def ant_colony_optimization(self,n_ants:int=50, n_iterations:int=1000, alpha:float=1, beta:float=1, evaporation_rate:float=0.9, Q:float=1):

        best_solution_length = float('inf')
        best_solution=None
        for iteration in range(n_iterations):
            # here we keep track of the paths of each ant in a given iteration
            solutions = []
            solution_lengths = []


            for ant in range(n_ants):

                # Initialize the ant's path
                depot=0
                visited = [False]*self.n_points
                # randomly select the starting point
                car_current_points = [depot for i in range(self.n_vehicules)]
                # set current point as visited
                visited[depot] = True
                # include the current point in the path
                car_paths = [[depot]for i in range(self.n_vehicules)]

                car_capacities=[self.capacity for i in range(self.n_vehicules)]
                # initialize the path length (cost)
                path_length = 0

                probabilities = np.zeros((self.n_vehicules,self.n_points))
                for i in range(self.n_vehicules):
                        # initialize the probabilities of moving to each unvisited node
                    while False in visited:  # while not all nodes are visited
                        # calculate the probabilities of visiting each node
                        has_unvisited=False
                        for candidate in range(self.n_points):
                            if (car_capacities[i]-self.demande[candidate]>=0) and candidate!=car_current_points[i] and not visited[candidate]:
                                has_unvisited=True
                                probabilities[i][candidate] = self.pheromone[car_current_points[i], candidate]**alpha / self.graph[car_current_points[i], candidate]**beta
                            else:
                                probabilities[i][candidate]=0
                        probabilities[i]=np.nan_to_num(probabilities[i])
                        if np.sum(probabilities[i])==0 or not has_unvisited:
                            break

                        probabilities[i] /= np.sum(probabilities[i])

                        car_i_next_point = np.random.choice(self.n_points, p=probabilities[i])
                        probabilities[:,car_i_next_point]=0
                        visited[car_i_next_point] = True
                        car_paths[i].append(car_i_next_point)

                        path_length += self.graph[car_current_points[i], car_i_next_point]
                        car_current_points[i]=car_i_next_point
                        car_capacities[i]-=self.demande[car_i_next_point]

                        #print(car_i_next_point)


                # the current ant has completed its path (visited all nodes)
                # so we append the path and its cost to the paths and path_lengths lists
                solutions.append([car_paths])
                solution_lengths.append(path_length)
                # we update the global best progress

                if path_length < best_solution_length:
                    best_solution = car_paths
                    best_solution_length = path_length
                #print(f'Iteration {iteration+1}/{n_iterations} - Best path length: {best_solution_length} - current path length: {path_length}')


            # update the pheromone matrix after all ants have completed their paths
            self.pheromone *= evaporation_rate

            for solution, solution_length in zip(solutions, solution_lengths):
                for sub_path in solution[0] :
                    for i in range(len(sub_path)-1):
                        self.pheromone[sub_path[i], sub_path[i+1]] += Q/solution_length
                    self.pheromone[sub_path[-1], sub_path[0]] += Q/solution_length


        return best_solution, best_solution_length

import numpy as np


data=np.matrix( [
    [0, 13, 6, 55, 93, 164, 166, 168, 169, 241, 212],
    [13, 0, 11, 66, 261, 175, 177, 179, 180, 239, 208],
    [6, 11, 0, 60, 97, 168, 171, 173, 174, 239, 209],
    [55, 66, 60, 0, 82, 113, 115, 117, 117, 295, 265],
    [93, 261, 97, 82, 0, 113, 115, 117, 118, 333, 302],
    [164, 175, 168, 113, 113, 0, 6, 7, 4, 403, 374],
    [166, 177, 171, 115, 115, 6, 0, 8, 7, 406, 376],
    [168, 179, 173, 117, 117, 7, 8, 0, 3, 408, 378],
    [169, 180, 174, 117, 118, 4, 7, 3, 0, 409, 379],
    [241, 239, 239, 295, 333, 403, 406, 408, 409, 0, 46],
    [212, 208, 209, 265, 302, 374, 376, 378, 379, 46, 0]
])

cvpr = CVPR(data, demande=[0, 5, 20, 10, 20, 85, 65, 30, 20, 70, 30], n_vehicules=6, capacity=100)

best_params = cvpr.optimize_params(n_ants=50, evaporation_rate=0.9, Q=1)
print("Best Parameters:", best_params)
#print(cvpr.ant_colony_optimization(n_ants=50, n_iterations=10, alpha=0, beta=0.25, evaporation_rate=0.9, Q=1))

1
1023
2
988
3
947
4
913
5
6
840
7
8
9
10
11
12
13
14
15
833
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271